In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
from random import shuffle
from tqdm import tqdm

In [ ]:
def haar_cascade_classifier(image):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    face = face_classifier.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
    
    if len(face) == 0:
        return None
    for (x, y, w, h) in face:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 4)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return img_rgb

In [ ]:
def create_train_data(TRAIN_DIR, LABEL_DIR):
    c0=0
    c1=0
    res=0
    
    training_data = []
    label_data = []
    
    for folder in os.listdir( LABEL_DIR ):
        c0=0
        
        for inner_folder in os.listdir( (LABEL_DIR+'/'+folder) ):
            path = os.path.join(LABEL_DIR, folder+'/'+inner_folder)
            
            for text_file in os.listdir( (path) ):
                text_file = os.path.join(path+'/'+text_file)
                with open(text_file) as opened_file:
                    lines = opened_file.readlines()
                    if(lines[0][3]=='0' and c0<340):
                        label_data.append(0)
                        res+=1
                        c0+=1
                    if(lines[0][3]!='0'):
                        label_data.append(1)
                        c1+=1
                    opened_file.close()
    
    label_data.append(-1)
    print(str(res)+ ' '+str(c1))
    
    c=0
    c0=0
    c1=0
    res=0
    print(len(label_data))
    
    for folder in os.listdir(TRAIN_DIR):
        c0=0
        for inner_folder in os.listdir( (TRAIN_DIR+'/'+folder) ):
            path = os.path.join(TRAIN_DIR, folder+'/'+inner_folder)
            for img in tqdm( os.listdir(path) ):
                img = os.path.join(path+'/'+img)
                img_data = cv2.imread(img, cv2.IMREAD_COLOR)
                #img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY)

                # PRE-PROCESSING & FACE DETECTION
                img_data = haar_cascade_classifier(img_data)
                if img_data is None:
                    continue
                
                #img_data = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
                
                if (label_data[c]==-1):break
                    
                # TRAINING_DATA = LISTOF([image_array, Label])
                if (label_data[c]==0 and c0<340):
                    training_data.append([np.array(img_data), label_data[c]])
                    c0+=1
                    res+=1
                    c+=1
                if (label_data[c]!=0):
                    training_data.append([np.array(img_data), label_data[c]])
                    c+=1
                    c1+=1
                    
    print(str(res)+' '+str(c1))
    shuffle(training_data)
    return training_data

In [ ]:
TRAIN_DIR = "/kaggle/input/emotionpain/Images"
LABEL_DIR = "/kaggle/input/emotionpain/Frame_Labels"


for fol in os.listdir(TRAIN_DIR):
    path1 = os.path.join(TRAIN_DIR+'/'+fol)
for folder in os.listdir(LABEL_DIR):
    path2 = os.path.join(LABEL_DIR+'/'+folder+'/PSPI')

In [ ]:
training_data = create_train_data(path1, path2)

In [ ]:
import matplotlib.pyplot as plt
mylist = [item[0] for item in training_data[70:100]]


images_per_row = 5
num_rows = len(mylist)//images_per_row + (1 if len(mylist)%images_per_row else 0)

plt.figure(figsize=(20, 4*num_rows)) # figsize = (Width, height) in inches
for i,img in enumerate(mylist):
    plt.subplot(num_rows, images_per_row, i+1)
    plt.imshow(img)
    plt.title(f'Image {i}')
    plt.axis('off')
    
plt.tight_layout()
plt.show()

from tensorflow.keras.callbacks import TensorBoard

import time

NAME = "pain-classifier-6Blocks-{}".format(int( time.time() ))

tensorboard = TensorBoard( log_dir="logs/{}".format(NAME) )

model.fit(X, y, epochs=3, validation_split=0.3, batch_size=16, callbacks=[tensorboard]) 

%%cmd
tensorboard --logdir=logs/

In [ ]:
import pickle

pickle_out = open("train.pickle","wb")
pickle.dump(training_data, pickle_out)
pickle_out.close()

# CNN Architecture Code

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Conv2D, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import cast
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pickle

In [ ]:
pickle_in = open("train.pickle","rb")
train_data = pickle.load(pickle_in)
len(training_data)

In [ ]:
import time
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
NAME = "Pain-classification-COLOR-{}".format( int(time.time()) )
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
def normalize_image(image):
    image = cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

In [ ]:
filtered_training_data = [item for item in training_data ]

labels = [item[1] for item in filtered_training_data]

images = [np.array(item[0], dtype=np.uint8) for item in filtered_training_data if item is not None]

In [ ]:

img_data = [cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR) for img in images]

images_array = np.array(img_data)
labels_array = np.array(labels)

images_array = normalize_image(images_array)

In [ ]:
labels_array_one_hot = to_categorical(labels_array, num_classes=2)
labels_array_one_hot

In [ ]:
no_of_filters = [32,64,96,96]

model = Sequential()

#Block 1
model.add( Conv2D( filters = 32, kernel_size=(3,3), input_shape=(96,96,3) ) )
model.add( BatchNormalization() )
model.add( Activation("relu") )
model.add( MaxPooling2D(pool_size=(2,2)) )
model.add( Dropout(0.2) )

for i in no_of_filters:
    model.add( Conv2D( filters = i, kernel_size=(3,3) ) )
    model.add( BatchNormalization() )
    model.add( Activation("relu") )
    model.add( MaxPooling2D( pool_size=(2,2) ) )
    model.add( Dropout(0.2) )
    
model.add( Conv2D( filters = 64, kernel_size=(3,3), padding='SAME' ) )
model.add( BatchNormalization() )
model.add( Activation("relu") )
model.add( MaxPooling2D( pool_size=(2,2), padding='SAME' ) )
model.add( Dropout(0.2) )

# Flatten
model.add( Flatten() )

model.add( Dense(256)   )
model.add( BatchNormalization() )
model.add( Activation("relu") )
model.add( Dropout(0.2) )
model.add( Dense(256) )

model.add( BatchNormalization() )
model.add( Activation("relu") )
model.add( Dropout(0.2) )
#model.add( Dense(7) )

# Output Layer
model.add( Dense(2, activation='softmax') )

In [ ]:
model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

In [ ]:
model.fit(images_array, labels_array_one_hot, batch_size=16, epochs=27, validation_split=0.3, callbacks=[tensorboard])

In [ ]:
import zipfile
import os

folder_path = 'logs'  # Folder you want to zip
zip_path = 'logs.zip'  # Output ZIP file name

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, os.path.join(folder_path, '..')))


In [ ]:
model.save('/kaggle/working/SAS.h5')

In [ ]:
model.save('SAS.keras')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/
